In [4]:
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import pickle
from pathlib import Path
from SPCA import helpers
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Rectangle
import matplotlib.patches as patches


In [5]:
# Set the mainpath, planet, channels, path, and other variables
rootpath = '/home/ldang05/projects/def-ncowan/ldang05/Spitzer_Data/'

# planet = 'WASP-34b'
# planet = 'WASP-52b'
# planet = 'WASP-74b'
# planet = 'WASP-76b'
# planet = 'WASP-77Ab'
# planet = 'WASP-95b'
# planet = 'WASP-140b'
# planet = 'KELT-7b' # BAD
# planet = 'KELT-14b'
# planet = 'KELT-20b'
# planet = 'TrES-3b'
planet = 'Qatar-2b'

channels = 'ch2'

if planet in ['HAT-P-23b', 'Qatar-2b', 'TrES-3b', 'WASP-76b']:
    ignore_no = 'ignore'
else: 
    ignore_no = 'noIgnore'

mainpath = rootpath + planet + '/analysis/' + channels + '/addedBlank/' + ignore_no + '/'

rows = ['v1_fixorbit', 'v1_PSFW_fixorbit', 'v2_fixorbit', 'v2_fixorbit_TruePSFW', 
        'v1_autoRun', 'v1_PSFW_autoRun', 'v2_autoRun', 'v2_PSFW_autoRun'] # for other planet
# fixorbit: only PLD and BLISS (no polynomial)
cols = ['Poly2', 'Poly3', 'Poly4', 'Poly5', 'BLISS', 'PLDAper1_3x3', 'PLDAper1_5x5']


minflux = 0.977
maxflux = 1.02

if planet == 'WASP-140b':
    minflux = 0.977
    maxflux = 1.02
    color = ''
elif planet == 'WASP-76b':
    minflux = 0.986
    maxflux = 1.019
    color = ''
elif planet == 'WASP-77Ab':
    minflux = 0.983
    maxflux = 1.018
    color = ''
elif planet == 'WASP-34b':
    minflux = 0.984
    maxflux = 1.02
    color = 'emerald_green'
elif planet == 'WASP-52b' or 'TrES-3b':
    minflux = 0.971
    maxflux = 1.01
    color = ''
elif planet == 'KELT-20b':
    rows = ['v1_TruePSFW', 'v1_PSFW_TruePSFW', 'v2_TruePSFW', 'v2_PSFW_TruePSFW'] # KELT-20b
    cols = ['Poly2', 'Poly3', 'Poly4', 'Poly5', 'BLISS', 'PLDAper1_3x3', 'PLDAper1_5x5']
    minflux = 0.985
    maxflux = 1.02
    color = ''


In [6]:
def create_subplot_names(rows, cols):
    # Create a dictionary to store subplot axes and their names
    subplot_names = {}

    # Create subplots with increased spacing
    fig, axes = plt.subplots(nrows=len(rows), ncols=len(cols), sharex=True, sharey=True, figsize=(30, len(rows) * 3))

    # Adjust the spacing between subplots
    plt.subplots_adjust(wspace=0, hspace=0)  # Adjust the values as needed
    plt.ylim(minflux, maxflux)

    # Iterate through rows and columns to name the subplots
    for i, row in enumerate(rows):
        for j, col in enumerate(cols):
            subplot_name = f"{col}_{row}"  # Switched order to col_row
            ax = axes[i, j]
            subplot_names[subplot_name] = ax

            # Set the title in the middle of each subplot
            # ax.set_title(subplot_name, fontsize=15, x=0.5, y=0.5, loc="center")  # Adjust the fontsize as needed

            # Set row names as labels for y axes for the first column
            if j == 0:
                ax.set_ylabel(row, fontsize=20, labelpad=20)  # Row name as y-axis label

    # Set column names at the top of the subplot (not as titles)
    for j, col in enumerate(cols):
        ax = axes[0, j]  # Access the top row of subplots
        ax.xaxis.set_label_position('top')  # Move the x-axis label to the top
        ax.set_xlabel(col, fontsize=20, labelpad=20)  # Column name as x-axis label

    return fig, subplot_names


# Define a custom exception for "file not found"
class FileNotFoundError(Exception):
    pass

def get_bic_values_dict(path):
    bic_values_dict = {}

    contents = os.listdir(path)

    for dir_4um in contents:
        dir_path = os.path.join(path, dir_4um)

        if os.path.isdir(dir_path):
            mode_folders = os.listdir(dir_path)

            for mode_folder in mode_folders:
                if mode_folder in mode_list:
                    mode_path = os.path.join(dir_path, mode_folder)

                    evidence_files = glob.glob(os.path.join(mode_path, 'EVIDENCE_*.txt'))

                    try:
                        # Check if evidence_files exist
                        if evidence_files:
                            evidence_file = evidence_files[0]

                            with open(evidence_file, 'r') as file:
                                evidence = file.read()
                                evidence_lines = evidence.split('\n')
                                last_line = evidence_lines[-1]
                                words = last_line.split()
                                bic = float(words[-1])
                                bic_values_dict[mode_folder] = bic

                        else:
                            # Raise a custom "file not found" exception
                            raise FileNotFoundError("EVIDENCE file not found for mode_folder: " + mode_folder)

                    except FileNotFoundError as e:
                        print(e)  # Print the error message
                        # Continue executing the rest of the code

    return bic_values_dict



def calculate_delta_bic(bic_values_dict):
    # Get the list of BIC values

    min_bic_key = min(bic_values_dict, key=lambda k: bic_values_dict[k])

    bic_values = list(bic_values_dict.values())
    min_bic = min(bic_values) # Find the minimum BIC value

    # Calculate delta BIC by subtracting the minimum BIC from each BIC value
    delta_bic_values = [bic - min_bic for bic in bic_values]

    # Create a dictionary that maps subplot names to their delta BIC values
    delta_bic_dict = {subplot_name: delta_bic for subplot_name, delta_bic in zip(bic_values_dict.keys(), delta_bic_values)}

    return delta_bic_dict, min_bic_key

def colormap(color=None):
    if color == "rosybrown":
        cmap = LinearSegmentedColormap.from_list('custom_rosybrown', [(0.737, 0.561, 0.561, 0.7), (0.737, 0.561, 0.561, 0)], N=256)
    elif color == "mintgreen":
        cmap = LinearSegmentedColormap.from_list('custom_mintgreen', [(0.678, 0.902, 0.749, 0.8), (0.678, 0.902, 0.749, 0)], N=256)
    elif color == "peach":
        cmap = LinearSegmentedColormap.from_list('custom_peach', [(0.976, 0.729, 0.651, 0.7), (0.976, 0.729, 0.651, 0)], N=256)
    elif color == "macaron_blue":
        cmap = LinearSegmentedColormap.from_list('custom_macaron_blue', [(0.482, 0.686, 0.937, 0.7), (0.482, 0.686, 0.937, 0)], N=256)
    elif color == "amber_orange":
        cmap = LinearSegmentedColormap.from_list('custom_amber_orange', [(1.0, 0.749, 0.0, 0.7), (1.0, 0.749, 0.0, 0)], N=256)
    elif color == "crimson_red":
        cmap = LinearSegmentedColormap.from_list('custom_crimson_red', [(0.863, 0.078, 0.235, 0.7), (0.863, 0.078, 0.235, 0)], N=256)
    elif color == "dark_teal":
        cmap = LinearSegmentedColormap.from_list('custom_dark_teal', [(0.0, 0.204, 0.204, 0.5), (0.0, 0.204, 0.204, 0)], N=256)
    elif color == "terracotta":
        cmap = LinearSegmentedColormap.from_list('custom_terracotta', [(0.796, 0.388, 0.286, 0.7), (0.796, 0.388, 0.286, 0)], N=256)
    elif color == "muted_lavender":
        cmap = LinearSegmentedColormap.from_list('custom_muted_lavender', [(0.655, 0.529, 0.714, 0.6), (0.655, 0.529, 0.714, 0)], N=256)
    else:
        cmap = LinearSegmentedColormap.from_list('custom_crimson_red', [(0.863, 0.078, 0.235, 0.6), (0.863, 0.078, 0.235, 0)], N=256)

    return cmap


def assign_colors_to_bic(bic_values_dict):
    bic_values = list(bic_values_dict.values())

    cmap = colormap(color)

    # Normalize BIC values to [0, 1] for colormap
    norm = plt.Normalize(min(bic_values), max(bic_values))
    
    # Map BIC values to colors
    colors = cmap(norm(bic_values))
    
    # Create a dictionary that maps subplot names to their colors
    bic_colors_dict = {subplot_name: color for subplot_name, color in zip(bic_values_dict.keys(), colors)}
    
    return bic_colors_dict



def plot_model(time, flux, astro, detec, breaks, 
               axName=None, bic=None,
               savepath=None, plotName='Combined_plot.pdf', plotTrueAnomaly=False, nbin=None, background_colors=None, min_subplot=False):   

    mcmc_signal = astro * detec
    
    if plotTrueAnomaly:
        x = time
    else:
        x = time - 5e4
    
    if nbin is not None:
        x_binned, _ = helpers.binValues(x, x, nbin)
        calibrated_binned, calibrated_binned_err = helpers.binValues(flux / detec, x, nbin, assumeWhiteNoise=True)
        residuals_binned, residuals_binned_err = helpers.binValues(flux / detec - astro, x, nbin, assumeWhiteNoise=True)
    
    # fig, axes = plt.subplots()
    axes = subplot_dict[axName]
    print('Mode {} plotted'.format(axName))

    # axes.plot(x, flux / detec, '.', color='k', markersize=4, alpha=0.15)
    axes.plot(x, astro, color='k', linewidth=2, alpha=0.7, zorder=100)
    if nbin is not None:
        axes.errorbar(x_binned, calibrated_binned, yerr=calibrated_binned_err, fmt='o',
                      color='grey', markersize=3, alpha=0.3)

    if bic is not None:
        axes.text(0.5, 0.80, r'$\Delta$BIC: {:.2f}'.format(bic), transform=axes.transAxes,
                  fontsize=18, ha='center', va='bottom', bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.5))

    axes.set_xticks([])
    axes.set_facecolor(background_colors)


    if min_subplot:
        pos = axes.get_position()
        # Bbox(x0=0.5678571428571428, y0=0.495, x1=0.6785714285714286, y1=0.6875)
   
        macaron_red = (0.929, 0.235, 0.443)
        goldenrod_yellow = (0.855, 0.647, 0.125)  # RGB values

        for spine in axes.spines.values():
            spine.set_edgecolor(goldenrod_yellow)  # Set the border color to red
            spine.set_linewidth(5)  # Set the border linewidth
            spine.set_zorder(100)

        # border_rect = plt.Rectangle((0, 0), 100, 100*pos.height,
        #                     linewidth=5, edgecolor=goldenrod_yellow,
        #                     facecolor='none', zorder=100)
        
        # axes.add_patch(Rectangle((0, 0), 100, 100,
        #      edgecolor = 'pink', facecolor = 'blue', fill=True, lw=5))


    return axes # x, astro, x_binned, calibrated_binned



fig, subplot_dict = create_subplot_names(rows, cols)
mode_list = list(subplot_dict.keys())

bic_values_dict = get_bic_values_dict(mainpath)
bic_values_dict, min_bic_key = calculate_delta_bic(bic_values_dict)

print('bic_values_dict is ', bic_values_dict)

bic_colors_dict = assign_colors_to_bic(bic_values_dict)
colors = list((assign_colors_to_bic(bic_values_dict)).values())


EVIDENCE file not found for mode_folder: Poly4_v2_autoRun
bic_values_dict is  {'BLISS_v1_PSFW_fixorbit': 4808.059966068133, 'BLISS_v1_autoRun': 3686.1606697840616, 'Poly2_v2_autoRun': 2014.6305273042526, 'Poly4_v1_autoRun': 2637.87352349516, 'BLISS_v1_fixorbit': 38.59572094393661, 'Poly3_v1_autoRun': 3863.9377412688336, 'Poly2_v1_autoRun': 2910.1470129381632, 'BLISS_v2_autoRun': 0.0, 'BLISS_v2_fixorbit': 968.2314379668096, 'Poly3_v2_autoRun': 3245.353663249174, 'PLDAper1_3x3_v2_autoRun': 24263.94677717873, 'PLDAper1_3x3_v2_fixorbit': 24434.249850110093, 'PLDAper1_3x3_v1_autoRun': 24846.82712705061, 'PLDAper1_3x3_v1_fixorbit': 24683.742437265406}


In [7]:
found_modes = set()

# List all files and directories in the specified path
contents = os.listdir(mainpath)

# Iterate through the contents and print their names
for dir_4um in contents:
    dir_path = os.path.join(mainpath, dir_4um)
    
    if os.path.isdir(dir_path):

        mode_folders = os.listdir(dir_path)

        for mode_folder in mode_folders:
            print('mode_folder is ', mode_folder)
            if mode_folder in mode_list:
                found_modes.add(mode_folder)
                mode_path = os.path.join(dir_path, mode_folder)

                
                bestfit_file = glob.glob(os.path.join(mode_path, 'Bestfit_*.pkl'))

                try:
                    if bestfit_file:

                        with open(bestfit_file[0], 'rb') as file:
                            bestfit_data = pickle.load(file)

                        # Extract data from loaded_data
                        time = bestfit_data[1]
                        flux = bestfit_data[2]
                        astro = bestfit_data[3]
                        detec = bestfit_data[4]
                        breaks = bestfit_data[5]

                        mode_folder = os.path.basename(mode_path)
                        bic = bic_values_dict.get(mode_folder, 0)  # Get the BIC value from the dictionary or use 0 if not found
                        background_colors = bic_colors_dict.get(mode_folder, 'white')  # Get background color or use 'white' if not found

                        if mode_folder == min_bic_key:
                            axes = plot_model(time, flux, astro, detec, breaks, axName=mode_folder, nbin=300, bic=bic, background_colors=background_colors, min_subplot=True)
                        else:
                            axes = plot_model(time, flux, astro, detec, breaks, axName=mode_folder, nbin=300, bic=bic, background_colors=background_colors)
                        # axes.set_xticks([])  # Hide x-axis ticks and values
                        # axes.set_facecolor(background_colors)

                    else:
                        raise FileNotFoundError("Bestfit file not found for mode_path: " + mode_path)

                except FileNotFoundError as e:
                        print(e)  # Print the error message
                        # Continue executing the rest of the code


# Save the figure to a PDF file
save_directory = Path("Compare_fits") # Define the directory where you want to save the figure    
save_directory.mkdir(parents=True, exist_ok=True) # Make sure the directory exists; create it if it doesn't
filename = 'combined_plot_{}.pdf'.format(planet) # Define the filename including the planets variable
save_path = save_directory / filename
print('Saving figure to', save_path)
plt.savefig(save_path, bbox_inches='tight')

not_found_modes = set(mode_list) - found_modes
print('Modes not found in mode_folder:', ' '.join(not_found_modes))

mode_folder is  BLISS_v1_PSFW_fixorbit
Mode BLISS_v1_PSFW_fixorbit plotted
mode_folder is  BLISS_v1_autoRun
Mode BLISS_v1_autoRun plotted
mode_folder is  Poly2_v2_autoRun
Mode Poly2_v2_autoRun plotted
mode_folder is  Poly4_v1_autoRun
Mode Poly4_v1_autoRun plotted
mode_folder is  ._.DS_Store
mode_folder is  .DS_Store
mode_folder is  BLISS_v1_fixorbit
Mode BLISS_v1_fixorbit plotted
mode_folder is  Poly3_v1_autoRun
Mode Poly3_v1_autoRun plotted
mode_folder is  Poly2_v1_autoRun
Mode Poly2_v1_autoRun plotted
mode_folder is  BLISS_v2_autoRun
Mode BLISS_v2_autoRun plotted
mode_folder is  BLISS_v2_PSFW_fixorbit
mode_folder is  BLISS_v2_fixorbit
Mode BLISS_v2_fixorbit plotted
mode_folder is  Poly2_v1
mode_folder is  Poly3_v2_autoRun
Mode Poly3_v2_autoRun plotted
mode_folder is  ch2_datacube_binned_AORs6152.dat
mode_folder is  ._ch2_datacube_binned_AORs6152.dat
mode_folder is  Lightcurve.pdf
mode_folder is  Poly4_v2_autoRun
Bestfit file not found for mode_path: /home/ldang05/projects/def-ncowan/